In [151]:
import pandas as pd
import os
from transformers import GPT2TokenizerFast, GPT2LMHeadModel, pipeline
import math
import torch
import numpy as np

input_dir = '../data/inputs'
output_dir = '../data/outputs'
results_dir = '../data/results'
if not os.path.exists(results_dir):
    os.makedirs(results_dir)

In [66]:


female_ratios = pd.read_csv(os.path.join(input_dir, 'female_ratios.csv'))
# for model in  llama3-8b llama3-8b-instruct mistral-7b mistral-7b-instruct llama2-7b llama2-7b-chat
model_strs = ['llama3-8b', 'llama3-8b-instruct', 'mistral-7b', 'mistral-7b-instruct', 'llama2-7b', 'llama2-7b-chat']
model_strs = sorted(model_strs, key=len, reverse=True)

prompt_ids = ['none','low-1','low-2','medium-3','medium-4','high-5','high-6']
prompt_id_mapping = {pid: idx for idx, pid in enumerate(prompt_ids)}

df_list = []
for f in os.listdir(output_dir):
    if "conv" not in f and "gender" not in f:
    # if 'conversation.csv' in f:
        
        df = pd.read_csv(os.path.join(output_dir, f))
    # if 'conversation.csv' in f:
        df = pd.read_csv(os.path.join(output_dir, f))
        df = pd.merge(df,female_ratios,on='job')
        df = df.drop(columns=['job','Unnamed: 0'])

        df['female_dominated'] = df['female_ratio'] > 50
        # Extract prompt ID from filename
        prompt_id_str = next((pid for pid in prompt_ids if pid in f), 'none')
        prompt_id = prompt_id_mapping[prompt_id_str]
        df['prompt_id'] = prompt_id
        
                # Extract model from filename
        model_str = next((model for model in model_strs if model in f), None)
        df['model'] = model_str
        df['conversation'] = 'conv' in f

        # Remove model name from other column names
        if model_str:
            df = df.rename(columns=lambda x: x.replace(f'{model_str}_', '') if model_str in x else x)

        
        # grouped_df = df.groupby('female_dominated').mean().reset_index()
        numeric_cols = df.select_dtypes(include='number').columns
        grouped_df = df.groupby(['female_dominated', 'model','conversation'])[numeric_cols].mean().reset_index()
    
        if 'male_met-met' in df.columns:
            col_ends = ['met-met', 'friend', 'talk-met']
            # Compute averages for male, female, and diverse columns
            male_cols = ['male_' + end for end in col_ends]
            female_cols = ['female_' + end for end in col_ends]
            diverse_cols = ['diverse_' + end for end in col_ends]

    
            grouped_df['male'] = grouped_df[male_cols].mean(axis=1)
            grouped_df['female'] = grouped_df[female_cols].mean(axis=1)
            grouped_df['diverse'] = grouped_df[diverse_cols].mean(axis=1)
            # make sure the three columns add up to 1
            # grouped_df['total'] = grouped_df['male'] + grouped_df['female'] + grouped_df['diverse']
            # 
            # Optionally, drop the original detailed columns if only averages are needed
            grouped_df = grouped_df.drop(columns=male_cols + female_cols + diverse_cols)

            
        df_list.append(grouped_df)
    # concat all the dataframes
df = pd.concat(df_list)
df.to_csv(os.path.join(results_dir, 'plain.csv'), index=False)
    


        

        

llama3-8b-instruct_none.csv
mistral-7b_low-1.csv
llama2-7b_none.csv
mistral-7b_low-2.csv
llama3-8b_none.csv
llama2-7b_medium-4.csv
mistral-7b_high-5.csv
llama3-8b-instruct_low-1.csv
llama2-7b-chat_high-5.csv
llama2-7b-chat_medium-4.csv
mistral-7b_high-6.csv
llama3-8b-instruct_low-2.csv
llama2-7b-chat_high-6.csv
mistral-7b-instruct_none.csv
mistral-7b_none.csv
llama2-7b-chat_low-2.csv
llama3-8b_high-6.csv
llama2-7b-chat_medium-3.csv
llama2-7b_medium-3.csv
llama2-7b-chat_low-1.csv
llama3-8b_high-5.csv
mistral-7b-instruct_low-1.csv
llama3-8b-instruct_high-6.csv
llama3-8b_low-1.csv
mistral-7b-instruct_low-2.csv
mistral-7b-instruct_medium-3.csv
llama3-8b-instruct_high-5.csv
llama3-8b_medium-3.csv
llama3-8b_low-2.csv
mistral-7b-instruct_high-5.csv
mistral-7b-instruct_high-6.csv
llama3-8b_medium-4.csv
mistral-7b-instruct_medium-4.csv
llama2-7b_high-5.csv
mistral-7b_medium-3.csv
llama2-7b_high-6.csv
llama3-8b-instruct_medium-4.csv
llama2-7b_low-2.csv
mistral-7b_medium-4.csv
llama2-7b_low-1.csv

In [68]:
df_list = []
for f in os.listdir(output_dir):
    if 'conversation.csv' in f:
        df = pd.read_csv(os.path.join(output_dir, f))
        df = pd.merge(df,female_ratios,on='job')
        df = df.drop(columns=['job','Unnamed: 0'])

        df['female_dominated'] = df['female_ratio'] > 50
        # Extract prompt ID from filename
        prompt_id_str = next((pid for pid in prompt_ids if pid in f), 'none')
        prompt_id = prompt_id_mapping[prompt_id_str]
        df['prompt_id'] = prompt_id
        
                # Extract model from filename
        model_str = next((model for model in model_strs if model in f), None)
        df['model'] = model_str
        df['conversation'] = 'conv' in f

        # Remove model name from other column names
        if model_str:
            df = df.rename(columns=lambda x: x.replace(f'{model_str}_', '') if model_str in x else x)

        
        # grouped_df = df.groupby('female_dominated').mean().reset_index()
        numeric_cols = df.select_dtypes(include='number').columns
        grouped_df = df.groupby(['female_dominated', 'model','conversation'])[numeric_cols].mean().reset_index()
    
        if 'male_met-met' in df.columns:
            col_ends = ['met-met', 'friend', 'talk-met']
            # Compute averages for male, female, and diverse columns
            male_cols = ['male_' + end for end in col_ends]
            female_cols = ['female_' + end for end in col_ends]
            diverse_cols = ['diverse_' + end for end in col_ends]

    
            grouped_df['male'] = grouped_df[male_cols].mean(axis=1)
            grouped_df['female'] = grouped_df[female_cols].mean(axis=1)
            grouped_df['diverse'] = grouped_df[diverse_cols].mean(axis=1)
            # make sure the three columns add up to 1
            # grouped_df['total'] = grouped_df['male'] + grouped_df['female'] + grouped_df['diverse']
            # 
            # Optionally, drop the original detailed columns if only averages are needed
            grouped_df = grouped_df.drop(columns=male_cols + female_cols + diverse_cols)

            
        df_list.append(grouped_df)
    # concat all the dataframes
df = pd.concat(df_list)
df.to_csv(os.path.join(results_dir, 'conversation.csv'), index=False)
    

llama2-7b-chat_medium-3_conversation.csv
llama2-7b_high-6_conversation.csv
mistral-7b-instruct_medium-3_conversation.csv
mistral-7b_low-2_conversation.csv
llama3-8b_high-5_conversation.csv
llama3-8b-instruct_high-6_conversation.csv
mistral-7b_medium-3_conversation.csv
llama2-7b-chat_low-1_conversation.csv
llama2-7b_medium-4_conversation.csv
llama3-8b-instruct_low-1_conversation.csv
llama3-8b-instruct_medium-4_conversation.csv
mistral-7b-instruct_none_conversation.csv
llama3-8b_low-1_conversation.csv
llama2-7b-chat_high-6_conversation.csv
mistral-7b-instruct_low-2_conversation.csv
mistral-7b_high-5_conversation.csv
mistral-7b-instruct_high-6_conversation.csv
llama3-8b_medium-4_conversation.csv
llama2-7b_low-2_conversation.csv
llama2-7b-chat_high-5_conversation.csv
llama3-8b_low-2_conversation.csv
llama3-8b_medium-3_conversation.csv
mistral-7b-instruct_low-1_conversation.csv
mistral-7b_high-6_conversation.csv
mistral-7b_none_conversation.csv
llama2-7b-chat_none_conversation.csv
llama2-7b

In [70]:
df_list = []
for f in os.listdir(output_dir):
    if 'genderquestion.csv' in f:
        df = pd.read_csv(os.path.join(output_dir, f))
        df = pd.merge(df,female_ratios,on='job')
        df = df.drop(columns=['job','Unnamed: 0'])


        df['female_dominated'] = df['female_ratio'] > 50
        # Extract prompt ID from filename
        prompt_id_str = next((pid for pid in prompt_ids if pid in f), 'none')
        prompt_id = prompt_id_mapping[prompt_id_str]
        df['prompt_id'] = prompt_id
        
                # Extract model from filename
        model_str = next((model for model in model_strs if model in f), None)
        df['model'] = model_str
        df['conversation'] = 'conv' in f

        # Remove model name from other column names
        if model_str:
            df = df.rename(columns=lambda x: x.replace(f'{model_str}_', '') if model_str in x else x)

        
        # grouped_df = df.groupby('female_dominated').mean().reset_index()
        numeric_cols = df.select_dtypes(include='number').columns
        grouped_df = df.groupby(['female_dominated', 'model','conversation'])[numeric_cols].mean().reset_index()
    
        if 'male_met-met' in df.columns:
            col_ends = ['met-met', 'friend', 'talk-met']
            # Compute averages for male, female, and diverse columns
            male_cols = ['male_' + end for end in col_ends]
            female_cols = ['female_' + end for end in col_ends]
            diverse_cols = ['diverse_' + end for end in col_ends]

    
            grouped_df['male'] = grouped_df[male_cols].mean(axis=1)
            grouped_df['female'] = grouped_df[female_cols].mean(axis=1)
            grouped_df['diverse'] = grouped_df[diverse_cols].mean(axis=1)
            # make sure the three columns add up to 1
            # grouped_df['total'] = grouped_df['male'] + grouped_df['female'] + grouped_df['diverse']
            # 
            # Optionally, drop the original detailed columns if only averages are needed
            grouped_df = grouped_df.drop(columns=male_cols + female_cols + diverse_cols)

            
        df_list.append(grouped_df)
    # concat all the dataframes
df = pd.concat(df_list)
df.to_csv(os.path.join(results_dir, 'genderquestion.csv'), index=False)

llama2-7b_high-5_genderquestion.csv
mistral-7b_medium-4_genderquestion.csv
llama2-7b_low-2_genderquestion.csv
llama3-8b-instruct_high-5_genderquestion.csv
llama3-8b-instruct_low-2_genderquestion.csv
llama3-8b-instruct_high-6_genderquestion.csv
llama3-8b-instruct_low-1_genderquestion.csv
llama3-8b-instruct_none_genderquestion.csv
llama2-7b_low-1_genderquestion.csv
llama2-7b_high-6_genderquestion.csv
llama2-7b_none_genderquestion.csv
llama2-7b_medium-3_genderquestion.csv
llama3-8b_high-5_genderquestion.csv
llama3-8b_low-1_genderquestion.csv
mistral-7b-instruct_low-2_genderquestion.csv
llama2-7b-chat_medium-4_genderquestion.csv
mistral-7b-instruct_none_genderquestion.csv
llama3-8b-instruct_medium-3_genderquestion.csv
mistral-7b-instruct_low-1_genderquestion.csv
llama2-7b-chat_none_genderquestion.csv
mistral-7b-instruct_medium-3_genderquestion.csv
llama3-8b_low-2_genderquestion.csv
llama3-8b_high-6_genderquestion.csv
llama3-8b_medium-4_genderquestion.csv
mistral-7b_medium-3_genderquestion.

In [71]:
df_list = []
for f in os.listdir(output_dir):
    if 'genderquestion_conv.csv' in f:
        df = pd.read_csv(os.path.join(output_dir, f))
        df = pd.merge(df,female_ratios,on='job')
        df = df.drop(columns=['job','Unnamed: 0'])
         

        df['female_dominated'] = df['female_ratio'] > 50
        # Extract prompt ID from filename
        prompt_id_str = next((pid for pid in prompt_ids if pid in f), 'none')
        prompt_id = prompt_id_mapping[prompt_id_str]
        df['prompt_id'] = prompt_id
        
                # Extract model from filename
        model_str = next((model for model in model_strs if model in f), None)
        df['model'] = model_str
        df['conversation'] = 'conv' in f

        # Remove model name from other column names
        if model_str:
            df = df.rename(columns=lambda x: x.replace(f'{model_str}_', '') if model_str in x else x)

        
        # grouped_df = df.groupby('female_dominated').mean().reset_index()
        numeric_cols = df.select_dtypes(include='number').columns
        grouped_df = df.groupby(['female_dominated', 'model','conversation'])[numeric_cols].mean().reset_index()
    
        if 'male_met-met' in df.columns:
            col_ends = ['met-met', 'friend', 'talk-met']
            # Compute averages for male, female, and diverse columns
            male_cols = ['male_' + end for end in col_ends]
            female_cols = ['female_' + end for end in col_ends]
            diverse_cols = ['diverse_' + end for end in col_ends]

    
            grouped_df['male'] = grouped_df[male_cols].mean(axis=1)
            grouped_df['female'] = grouped_df[female_cols].mean(axis=1)
            grouped_df['diverse'] = grouped_df[diverse_cols].mean(axis=1)
            # make sure the three columns add up to 1
            # grouped_df['total'] = grouped_df['male'] + grouped_df['female'] + grouped_df['diverse']
            # 
            # Optionally, drop the original detailed columns if only averages are needed
            grouped_df = grouped_df.drop(columns=male_cols + female_cols + diverse_cols)

            
        df_list.append(grouped_df)
    # concat all the dataframes
df = pd.concat(df_list)
df.to_csv(os.path.join(results_dir, 'genderquestion_conv.csv'), index=False)

llama2-7b_high-6_genderquestion_conv.csv
mistral-7b_medium-3_genderquestion_conv.csv
mistral-7b_none_genderquestion_conv.csv
llama3-8b-instruct_high-6_genderquestion_conv.csv
llama3-8b_medium-4_genderquestion_conv.csv
mistral-7b_high-6_genderquestion_conv.csv
llama2-7b-chat_low-1_genderquestion_conv.csv
llama3-8b-instruct_medium-4_genderquestion_conv.csv
llama2-7b-chat_low-2_genderquestion_conv.csv
mistral-7b-instruct_none_genderquestion_conv.csv
llama3-8b-instruct_high-5_genderquestion_conv.csv
mistral-7b_high-5_genderquestion_conv.csv
mistral-7b_medium-4_genderquestion_conv.csv
llama3-8b-instruct_medium-3_genderquestion_conv.csv
llama3-8b_none_genderquestion_conv.csv
llama2-7b_high-5_genderquestion_conv.csv
llama3-8b_medium-3_genderquestion_conv.csv
llama2-7b-chat_none_genderquestion_conv.csv
mistral-7b-instruct_medium-3_genderquestion_conv.csv
llama3-8b-instruct_low-2_genderquestion_conv.csv
llama2-7b_medium-4_genderquestion_conv.csv
llama2-7b-chat_medium-3_genderquestion_conv.csv
m

In [103]:
df_gq = pd.read_csv(os.path.join(results_dir, 'plain.csv'))
df_gq_conv = pd.read_csv(os.path.join(results_dir, 'conversation.csv'))

df = pd.concat([df_gq, df_gq_conv])
for model, group_df in df.groupby('model'):
    # Save the DataFrame to a CSV file
    if not os.path.exists(os.path.join(results_dir,model)):
        os.makedirs(os.path.join(results_dir,model))
    group_df.to_csv(os.path.join(results_dir,model, f'non_gq.csv'), index=False)

    # Optional: Print a message to indicate the file has been saved
    # print(f"Saved {model} non_gq.csv")

Saved llama2-7b non_gq.csv
Saved llama2-7b-chat non_gq.csv
Saved llama3-8b non_gq.csv
Saved llama3-8b-instruct non_gq.csv
Saved mistral-7b non_gq.csv
Saved mistral-7b-instruct non_gq.csv


In [161]:
import os
import pandas as pd

# Define the mapping dictionaries
model_str_map = {
    'llama3-8b': 'Llama3 8B',
    'llama3-8b-instruct': 'Llama3 8B (Instruct)',
    'mistral-7b': 'Mistral 7B',
    'mistral-7b-instruct': 'Mistral 7B (Instruct)',
    'llama2-7b': 'LLAMA2 7B',
    'llama2-7b-chat': 'LLAMA2 7B (Chat)'
}

prompt_id_map = {
    0: 'None',
    1: '1',
    2: '2',
    3: '3',
    4: '4',
    5: '5',
    6: '6'
}

abstraction_levels = {
    0: '',
    1: 'High',
    2: 'High',
    3: 'Med.',
    4: 'Med.',
    5: 'Low',
    6: 'Low'
}

abs_order = ['', 'High', 'Med.', 'Low']

In [164]:
 # Update this to your results directory
models = list(model_str_map.keys())

output_tex_file = os.path.join(results_dir, 'aggregated_results.tex')

with open(output_tex_file, 'w') as f_out:
    for model in models:
        model_name = model.replace('-', '')
        model_description = model_str_map[model]

        f_out.write(r'\subsection{' + model_description + '}\n')

        for csv_file, (caption, label_suffix) in [('genderquestion.csv', ('Probability of gender expressions for task prompt number one. ``ID\'\' refers to the id of the debiasing prompt, ``Abs.\'\' indicates its degree of abstraction.', 'genderquestion')),
                                                  ('non_gq.csv', ('Aggregated results for task prompts two to four. ``ID\'\' refers to the debiasing prompt, ``Abs.\'\' indicates its degree of abstraction.', 'nongq'))]:
            
            
            # Load the CSV file into a DataFrame
            df_path = os.path.join(results_dir, model, csv_file)
            df = pd.read_csv(df_path)

            # Add the abstraction level column to the DataFrame
            df['Abs.'] = df['prompt_id'].map(abstraction_levels)

            # Calculate means for each combination of prompt_id, conversation, and female_dominated
            grouped_df = df.groupby(['Abs.', 'model', 'prompt_id', 'conversation', 'female_dominated']).mean(numeric_only=True).reset_index()
            model = grouped_df['model'].unique()[0]

            # Drop the 'model' column
            model_df = grouped_df.drop(columns=['model'])
                        # Sort by Abs. and prompt_id to ensure correct order
            model_df['prompt_id'] = model_df['prompt_id'].astype(int)
            model_df['Abs.'] = pd.Categorical(model_df['Abs.'], categories=abs_order, ordered=True)
            model_df = model_df.sort_values(by=['Abs.', 'prompt_id'])

            latex_table = r'''
\begin{table*}[ht!]
\centering
\small
    % Reduce text size and slightly the gap between columns
    \setlength{\tabcolsep}{4.6pt} % Default: 5pt
    % \resizebox{\textwidth}{!}{  % Alternative method: resize entire table (problem: also resizes line widths)
    \begin{tabular}{l c c c c c c c c c c c c c c}
    \toprule
    & & \multicolumn{6}{c}{Without Dialogue} & \multicolumn{6}{c}{With Dialogue} \\
    \cmidrule(lr){3-8} \cmidrule(lr){9-14}
    & & \multicolumn{3}{c}{Female Dominated} & \multicolumn{3}{c}{Male Dominated} & \multicolumn{3}{c}{Female Dominated} & \multicolumn{3}{c}{Male Dominated} \\
    \cmidrule(lr){3-5} \cmidrule(lr){6-8} \cmidrule(lr){9-11} \cmidrule(lr){12-14}
        Abs. & ID & M & F & D & M & F & D & M & F & D & M & F & D\\
        \midrule
'''

            # Sort by prompt_id to ensure order from None to 6
            model_df['prompt_id'] = model_df['prompt_id'].astype(int)
            model_df = model_df.sort_values(by=['Abs.','prompt_id'])

            previous_abs_level = None

            for abs_level, abs_group in model_df.groupby('Abs.'):
                if previous_abs_level is not None:
                    latex_table += "        \\midrule\n"
                for prompt_id, prompt_group in abs_group.groupby('prompt_id'):
                    id_label = 'None' if prompt_id == 0 else str(prompt_id)
                    row_prefix = f"        {abs_level if prompt_id in [0,2,4,6] else ''} & {id_label} & "

                    no_dialogue_fd = prompt_group[(prompt_group['conversation'] == False) & (prompt_group['female_dominated'] == True)]
                    no_dialogue_md = prompt_group[(prompt_group['conversation'] == False) & (prompt_group['female_dominated'] == False)]
                    dialogue_fd = prompt_group[(prompt_group['conversation'] == True) & (prompt_group['female_dominated'] == True)]
                    dialogue_md = prompt_group[(prompt_group['conversation'] == True) & (prompt_group['female_dominated'] == False)]

                    row = row_prefix

                    if not no_dialogue_fd.empty:
                        row += f"{no_dialogue_fd['male'].values[0]*100:.1f}\\% & {no_dialogue_fd['female'].values[0]*100:.1f}\\% & {no_dialogue_fd['diverse'].values[0]*100:.1f}\\% & "
                    else:
                        row += " & & & "

                    if not no_dialogue_md.empty:
                        row += f"{no_dialogue_md['male'].values[0]*100:.1f}\\% & {no_dialogue_md['female'].values[0]*100:.1f}\\% & {no_dialogue_md['diverse'].values[0]*100:.1f}\\% & "
                    else:
                        row += " & & & "

                    if not dialogue_fd.empty:
                        row += f"{dialogue_fd['male'].values[0]*100:.1f}\\% & {dialogue_fd['female'].values[0]*100:.1f}\\% & {dialogue_fd['diverse'].values[0]*100:.1f}\\% & "
                    else:
                        row += " & & & "

                    if not dialogue_md.empty:
                        row += f"{dialogue_md['male'].values[0]*100:.1f}\\% & {dialogue_md['female'].values[0]*100:.1f}\\% & {dialogue_md['diverse'].values[0]*100:.1f}\\% "
                    else:
                        row += " & & "

                    row += r"\\"
                    latex_table += row + "\n"

                if abs_level != '':  # Add averages for all except "None"
                    avg_row_prefix = f"         & \\textbf{{Avg}} & "

                    avg_no_dialogue_fd = abs_group[(abs_group['conversation'] == False) & (abs_group['female_dominated'] == True)][['male', 'female', 'diverse']].mean(numeric_only=True)
                    avg_no_dialogue_md = abs_group[(abs_group['conversation'] == False) & (abs_group['female_dominated'] == False)][['male', 'female', 'diverse']].mean(numeric_only=True)
                    avg_dialogue_fd = abs_group[(abs_group['conversation'] == True) & (abs_group['female_dominated'] == True)][['male', 'female', 'diverse']].mean(numeric_only=True)
                    avg_dialogue_md = abs_group[(abs_group['conversation'] == True) & (abs_group['female_dominated'] == False)][['male', 'female', 'diverse']].mean(numeric_only=True)

                    avg_row = avg_row_prefix

                    if not avg_no_dialogue_fd.empty:
                        avg_row += f"\\textbf{{{avg_no_dialogue_fd['male']*100:.1f}\\%}} & \\textbf{{{avg_no_dialogue_fd['female']*100:.1f}\\%}} & \\textbf{{{avg_no_dialogue_fd['diverse']*100:.1f}\\%}} & "
                    else:
                        avg_row += " & & & "

                    if not avg_no_dialogue_md.empty:
                        avg_row += f"\\textbf{{{avg_no_dialogue_md['male']*100:.1f}\\%}} & \\textbf{{{avg_no_dialogue_md['female']*100:.1f}\\%}} & \\textbf{{{avg_no_dialogue_md['diverse']*100:.1f}\\%}} & "
                    else:
                        avg_row += " & & & "

                    if not avg_dialogue_fd.empty:
                        avg_row += f"\\textbf{{{avg_dialogue_fd['male']*100:.1f}\\%}} & \\textbf{{{avg_dialogue_fd['female']*100:.1f}\\%}} & \\textbf{{{avg_dialogue_fd['diverse']*100:.1f}\\%}} & "
                    else:
                        avg_row += " & & & "

                    if not avg_dialogue_md.empty:
                        avg_row += f"\\textbf{{{avg_dialogue_md['male']*100:.1f}\\%}} & \\textbf{{{avg_dialogue_md['female']*100:.1f}\\%}} & \\textbf{{{avg_dialogue_md['diverse']*100:.1f}\\%}} "
                    else:
                        avg_row += " & & "

                    avg_row += r"\\"
                    latex_table += avg_row + "\n"

                previous_abs_level = abs_level

            latex_table += r'''
\bottomrule
\end{tabular}
% } % end \resizebox
\caption{''' + caption + r'''}
\label{tab:''' + label_suffix + r'''_''' + model_name + r'''}
\end{table*}

'''

            f_out.write(latex_table)
        f_out.write('\\clearpage\n\n')

print("Aggregated LaTeX table saved to", output_tex_file)

Aggregated LaTeX table saved to ../data/results/aggregated_results.tex


/var/folders/h3/mg_n_0ls7kgb1369wvkmb_dh0000gq/T/ipykernel_48870/1993601125.py:58: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for abs_level, abs_group in model_df.groupby('Abs.'):
/var/folders/h3/mg_n_0ls7kgb1369wvkmb_dh0000gq/T/ipykernel_48870/1993601125.py:58: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for abs_level, abs_group in model_df.groupby('Abs.'):
/var/folders/h3/mg_n_0ls7kgb1369wvkmb_dh0000gq/T/ipykernel_48870/1993601125.py:58: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or